# 🎨 간판 Pix2Pix 학습 노트북

**목적:** Phase 1 CG 이미지를 실제 사진처럼 변환하는 모델 학습

**데이터:** paired_data (INPUT: CG 간판, TARGET: 실제 사진)

---

## 📋 사전 준비

1. ✅ Google Drive에 데이터 업로드 완료 (`Colab_Signboard_Data` 폴더)
2. ✅ 이 노트북을 Google Colab에서 열기
3. ✅ GPU 런타임 사용 확인 (런타임 → 런타임 유형 변경 → GPU)

## 1️⃣ 환경 확인 및 GPU 설정

In [ ]:
# GPU 확인
import torch
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA 버전: {torch.version.cuda}")
else:
    print("⚠️ GPU가 감지되지 않습니다. 런타임을 GPU로 변경하세요.")

## 2️⃣ Google Drive 연결

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 데이터 경로 확인
import os
DATA_PATH = '/content/drive/MyDrive/Colab_Signboard_Data'

if os.path.exists(DATA_PATH):
    print(f"✅ 데이터 경로 확인: {DATA_PATH}")
    # 파일 개수 확인
    train_input_count = len([f for f in os.listdir(os.path.join(DATA_PATH, 'train', 'input')) if f.endswith(('.png', '.jpg'))])
    train_target_count = len([f for f in os.listdir(os.path.join(DATA_PATH, 'train', 'target')) if f.endswith(('.png', '.jpg'))])
    test_input_count = len([f for f in os.listdir(os.path.join(DATA_PATH, 'test', 'input')) if f.endswith(('.png', '.jpg'))])
    test_target_count = len([f for f in os.listdir(os.path.join(DATA_PATH, 'test', 'target')) if f.endswith(('.png', '.jpg'))])
    print(f"\n📊 데이터 통계:")
    print(f"  Train: {train_input_count} input, {train_target_count} target")
    print(f"  Test: {test_input_count} input, {test_target_count} target")
else:
    print(f"❌ 데이터 경로를 찾을 수 없습니다: {DATA_PATH}")
    print("\n📋 다음을 확인하세요:")
    print("  1. Google Drive에 'Colab_Signboard_Data' 폴더가 있는지")
    print("  2. 폴더 구조가 올바른지 (train/input, train/target, test/input, test/target)")

## 3️⃣ Pix2Pix 저장소 클론 및 설치

In [ ]:
# 작업 디렉토리로 이동
%cd /content

# Pix2Pix 저장소 클론 (이미 있으면 스킵)
if not os.path.exists('pytorch-CycleGAN-and-pix2pix'):
    !git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git
    print("✅ 저장소 클론 완료")
else:
    print("✅ 저장소가 이미 있습니다")

%cd pytorch-CycleGAN-and-pix2pix

# 필요한 라이브러리 설치
!pip install -q dominate visdom
print("✅ 필요한 라이브러리 설치 완료")

## 4️⃣ 데이터를 Pix2Pix 형식으로 준비

In [ ]:
# Pix2Pix 데이터 형식으로 변환
import shutil

DATASET_NAME = 'signboards'
DATASET_DIR = f'/content/pytorch-CycleGAN-and-pix2pix/datasets/{DATASET_NAME}'

# 디렉토리 생성
os.makedirs(f'{DATASET_DIR}/train/A', exist_ok=True)
os.makedirs(f'{DATASET_DIR}/train/B', exist_ok=True)
os.makedirs(f'{DATASET_DIR}/test/A', exist_ok=True)
os.makedirs(f'{DATASET_DIR}/test/B', exist_ok=True)

# 심볼릭 링크 생성 (복사 대신 링크 사용 - 빠름)
# Train
!ln -s "{DATA_PATH}/train/input/"* "{DATASET_DIR}/train/A/" 2>/dev/null || true
!ln -s "{DATA_PATH}/train/target/"* "{DATASET_DIR}/train/B/" 2>/dev/null || true

# Test
!ln -s "{DATA_PATH}/test/input/"* "{DATASET_DIR}/test/A/" 2>/dev/null || true
!ln -s "{DATA_PATH}/test/target/"* "{DATASET_DIR}/test/B/" 2>/dev/null || true

# 확인
train_a_count = len([f for f in os.listdir(f'{DATASET_DIR}/train/A') if not f.startswith('.')])
train_b_count = len([f for f in os.listdir(f'{DATASET_DIR}/train/B') if not f.startswith('.')])
test_a_count = len([f for f in os.listdir(f'{DATASET_DIR}/test/A') if not f.startswith('.')])
test_b_count = len([f for f in os.listdir(f'{DATASET_DIR}/test/B') if not f.startswith('.')])

print(f"✅ 데이터 준비 완료:")
print(f"  Train: A={train_a_count}, B={train_b_count}")
print(f"  Test: A={test_a_count}, B={test_b_count}")

## 5️⃣ 데이터 샘플 확인

In [ ]:
# 샘플 이미지 확인
from PIL import Image
import matplotlib.pyplot as plt

def show_sample(train_dir, sample_idx=0):
    a_files = sorted([f for f in os.listdir(f'{train_dir}/A') if f.endswith(('.png', '.jpg')) and not f.startswith('.')])
    b_files = sorted([f for f in os.listdir(f'{train_dir}/B') if f.endswith(('.png', '.jpg')) and not f.startswith('.')])
    
    if sample_idx < len(a_files) and sample_idx < len(b_files):
        a_path = os.path.join(train_dir, 'A', a_files[sample_idx])
        b_path = os.path.join(train_dir, 'B', b_files[sample_idx])
        
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        
        img_a = Image.open(a_path)
        axes[0].imshow(img_a)
        axes[0].set_title(f'INPUT (CG) - {a_files[sample_idx]}', fontsize=12)
        axes[0].axis('off')
        
        img_b = Image.open(b_path)
        axes[1].imshow(img_b)
        axes[1].set_title(f'TARGET (실제 사진) - {b_files[sample_idx]}', fontsize=12)
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
    else:
        print(f"❌ 샘플 인덱스 {sample_idx}를 찾을 수 없습니다.")

print("📸 Train 데이터 샘플:")
show_sample(f'{DATASET_DIR}/train', 0)

print("\n📸 Test 데이터 샘플:")
show_sample(f'{DATASET_DIR}/test', 0)

## 6️⃣ 학습 시작

In [ ]:
# 학습 파라미터 설정 (🚨 이미지 분할 문제 해결!)
MODEL_NAME = 'signboard_pix2pix_FIXED'  # 새 모델명
EPOCHS = 25  # 테스트를 위해 짧게 설정
EPOCHS_DECAY = 25  # Learning rate decay epoch
BATCH_SIZE = 1  # GPU 메모리에 따라 조정 (1, 2, 4 등)

print(f"🎯 수정된 학습 설정:")
print(f"  모델 이름: {MODEL_NAME}")
print(f"  Epochs: {EPOCHS} + {EPOCHS_DECAY} (decay)")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  데이터셋: {DATASET_NAME}")
print(f"\n⏱️  예상 소요 시간: 30분-1시간 (GPU 기준)")
print(f"\n💡 중간 결과는 checkpoints/{MODEL_NAME}/web/images/ 에 저장됩니다.")
print(f"\n🔧 이미지 분할 문제 해결을 위한 추가 옵션이 적용됩니다!")

In [ ]:
# 학습 실행 (🚨 이미지 분할 문제 완전 해결!)
!python train.py --dataroot ./datasets/{DATASET_NAME} --name {MODEL_NAME} --model pix2pix --direction AtoB --dataset_mode aligned --load_size 512 --crop_size 512 --preprocess none --no_flip --input_nc 3 --output_nc 3 --batch_size {BATCH_SIZE} --n_epochs {EPOCHS} --n_epochs_decay {EPOCHS_DECAY} --display_freq 25 --print_freq 25 --save_epoch_freq 5

print('🚨 이미지 분할 문제 해결 옵션들:')
print('  --dataset_mode aligned: 별도 A/B 폴더 사용')
print('  --load_size 512: 이미지를 512x512로 로드')
print('  --crop_size 512: 크롭 없이 전체 이미지 사용')
print('  --preprocess none: 추가 전처리 없음')
print('  --no_flip: 좌우 반전 비활성화 (핵심!)')
print('  --input_nc 3 --output_nc 3: 3채널 명시')
print('  --display_freq 25: 25 iter마다 결과 저장')
print('🎯 완전한 이미지로 학습합니다!')

## 7️⃣ 중간 결과 확인 (학습 중간에 실행 가능)

## ⚡ 빠른 확인 (Epoch 10에서)

**학습 중간에 이미지 분할 문제가 해결되었는지 빠르게 확인하세요!**


In [ ]:
# 🚨 중요! Epoch 10 정도에서 이 셀을 실행해서 분할 문제가 해결되었는지 확인하세요!
result_dir = f'/content/pytorch-CycleGAN-and-pix2pix/checkpoints/{MODEL_NAME}/web/images'

if os.path.exists(result_dir):
    # epoch010 확인
    epoch_dir = os.path.join(result_dir, 'epoch010')
    if os.path.exists(epoch_dir):
        print("✅ Epoch 10 결과 확인:")
        
        real_a_path = os.path.join(epoch_dir, 'real_A.png')
        fake_b_path = os.path.join(epoch_dir, 'fake_B.png') 
        real_b_path = os.path.join(epoch_dir, 'real_B.png')
        
        if all(os.path.exists(p) for p in [real_a_path, fake_b_path, real_b_path]):
            fig, axes = plt.subplots(1, 3, figsize=(18, 6))
            
            img_a = Image.open(real_a_path)
            axes[0].imshow(img_a)
            axes[0].set_title('INPUT (CG) - 완전한 간판이어야 함!', fontsize=14)
            axes[0].axis('off')
            print(f"Real_A 크기: {img_a.size}")
            
            img_fake = Image.open(fake_b_path)
            axes[1].imshow(img_fake) 
            axes[1].set_title('OUTPUT (생성) - 완전한 간판이어야 함!', fontsize=14)
            axes[1].axis('off')
            print(f"Fake_B 크기: {img_fake.size}")
            
            img_b = Image.open(real_b_path)
            axes[2].imshow(img_b)
            axes[2].set_title('TARGET (실제) - 완전한 간판이어야 함!', fontsize=14) 
            axes[2].axis('off')
            print(f"Real_B 크기: {img_b.size}")
            
            plt.tight_layout()
            plt.show()
            
            # 크기 확인
            if img_a.size == (512, 512) and img_b.size == (512, 512):
                print("\n🎉 성공! 이미지가 완전하게 나오고 있습니다!")
            else:
                print(f"\n❌ 문제! 이미지 크기가 이상합니다: A={img_a.size}, B={img_b.size}")
        else:
            print("❌ Epoch 10 이미지를 찾을 수 없습니다.")
    else:
        print("⏳ Epoch 10이 아직 완료되지 않았습니다. 잠시 기다려주세요...")
else:
    print("❌ 결과 폴더를 찾을 수 없습니다. 학습이 시작되었는지 확인하세요.")


In [ ]:
# 가장 최근 epoch의 결과 확인
result_dir = f'/content/pytorch-CycleGAN-and-pix2pix/checkpoints/{MODEL_NAME}/web/images'

if os.path.exists(result_dir):
    # 가장 최근 epoch 찾기 (디렉토리만, epoch + 숫자 패턴만)
    import re
    epoch_dirs = []
    for item in os.listdir(result_dir):
        item_path = os.path.join(result_dir, item)
        # epoch로 시작하고 디렉토리이며, 숫자로만 구성된 것만
        if item.startswith('epoch') and os.path.isdir(item_path):
            # epoch 뒤에 숫자만 있는지 확인
            match = re.match(r'^epoch(\d+)$', item)
            if match:
                epoch_dirs.append(item)
    
    if epoch_dirs:
        latest_epoch = max(epoch_dirs, key=lambda x: int(re.match(r'^epoch(\d+)$', x).group(1)))
        epoch_dir = os.path.join(result_dir, latest_epoch)
        
        # 결과 이미지 표시
        real_a_path = os.path.join(epoch_dir, 'real_A.png')
        fake_b_path = os.path.join(epoch_dir, 'fake_B.png')
        real_b_path = os.path.join(epoch_dir, 'real_B.png')
        
        if all(os.path.exists(p) for p in [real_a_path, fake_b_path, real_b_path]):
            fig, axes = plt.subplots(1, 3, figsize=(18, 6))
            
            axes[0].imshow(Image.open(real_a_path))
            axes[0].set_title('INPUT (CG)', fontsize=14)
            axes[0].axis('off')
            
            axes[1].imshow(Image.open(fake_b_path))
            axes[1].set_title(f'OUTPUT (생성됨) - Epoch {latest_epoch}', fontsize=14)
            axes[1].axis('off')
            
            axes[2].imshow(Image.open(real_b_path))
            axes[2].set_title('TARGET (실제)', fontsize=14)
            axes[2].axis('off')
            
            plt.tight_layout()
            plt.show()
        else:
            print(f"⚠️ {latest_epoch}의 이미지를 찾을 수 없습니다.")
    else:
        print("⚠️ 결과 이미지가 아직 생성되지 않았습니다.")
else:
    print(f"⚠️ 결과 디렉토리를 찾을 수 없습니다: {result_dir}")

## 8️⃣ 테스트 실행 (학습 완료 후)

In [ ]:
# 테스트 실행 (🚨 이미지 분할 문제 완전 해결!)
!python test.py --dataroot ./datasets/{DATASET_NAME} --name {MODEL_NAME} --model pix2pix --direction AtoB --dataset_mode aligned --load_size 512 --crop_size 512 --preprocess none --no_flip --input_nc 3 --output_nc 3 --epoch latest

print('🚨 테스트 옵션도 분할 문제 해결 적용!')
print('  --no_flip: 좌우 반전 비활성화')
print('  --input_nc 3 --output_nc 3: 3채널 명시')
print('🎯 완전한 이미지로 테스트합니다!')

## 9️⃣ 테스트 결과 확인

In [ ]:
# 테스트 결과 이미지 표시
test_result_dir = f'/content/pytorch-CycleGAN-and-pix2pix/results/{MODEL_NAME}/test_latest/images'

if os.path.exists(test_result_dir):
    test_files = sorted([f for f in os.listdir(test_result_dir) if f.endswith('.png')])
    
    # 첫 번째 샘플 표시
    if test_files:
        real_a_files = [f for f in test_files if 'real_A' in f]
        fake_b_files = [f for f in test_files if 'fake_B' in f]
        real_b_files = [f for f in test_files if 'real_B' in f]
        
        num_samples = min(3, len(real_a_files))  # 최대 3개 샘플
        
        for i in range(num_samples):
            fig, axes = plt.subplots(1, 3, figsize=(18, 6))
            
            axes[0].imshow(Image.open(os.path.join(test_result_dir, real_a_files[i])))
            axes[0].set_title('INPUT', fontsize=14)
            axes[0].axis('off')
            
            axes[1].imshow(Image.open(os.path.join(test_result_dir, fake_b_files[i])))
            axes[1].set_title('OUTPUT (모델 생성)', fontsize=14)
            axes[1].axis('off')
            
            axes[2].imshow(Image.open(os.path.join(test_result_dir, real_b_files[i])))
            axes[2].set_title('TARGET (실제)', fontsize=14)
            axes[2].axis('off')
            
            plt.tight_layout()
            plt.show()
    else:
        print("⚠️ 테스트 결과 파일을 찾을 수 없습니다.")
else:
    print(f"⚠️ 테스트 결과 디렉토리를 찾을 수 없습니다: {test_result_dir}")

## 🔟 모델 다운로드 (선택사항)

In [ ]:
# 학습된 모델을 Google Drive에 저장 (선택사항)
checkpoint_dir = f'/content/pytorch-CycleGAN-and-pix2pix/checkpoints/{MODEL_NAME}'
drive_save_dir = '/content/drive/MyDrive/Colab_Signboard_Models'

if os.path.exists(checkpoint_dir):
    os.makedirs(drive_save_dir, exist_ok=True)
    
    # 모델 파일 복사
    !cp -r "{checkpoint_dir}" "{drive_save_dir}/"
    
    print(f"✅ 모델이 {drive_save_dir}에 저장되었습니다.")
else:
    print(f"⚠️ 체크포인트 디렉토리를 찾을 수 없습니다: {checkpoint_dir}")